In [1]:
import pandas as pd

In [2]:
#mimiciv_dir = '/data/vision/polina/projects/chestxray/data_mimic_iv/0.4/'

In [3]:
admissions = pd.read_csv(f'{mimiciv_dir}core/admissions.csv', low_memory=False)
admissions[:10]

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,12427812,21593330,2184-01-06 11:51:00,2184-01-10 11:45:00,NaN,URGENT,PHYSICIAN REFERRAL,HOME,Other,ENGLISH,NaN,UNKNOWN,NaN,NaN,0
1,14029832,22059088,2120-01-18 01:28:00,2120-01-20 16:13:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,NaN,OTHER,NaN,NaN,0
2,14495017,22484010,2175-01-28 15:41:00,2175-01-29 16:00:00,NaN,DIRECT EMER.,PHYSICIAN REFERRAL,HOME,Other,?,NaN,WHITE,NaN,NaN,0
3,13676048,23865469,2193-01-19 05:27:00,2193-01-24 18:59:00,NaN,URGENT,PHYSICIAN REFERRAL,HOME,Other,?,MARRIED,WHITE,NaN,NaN,0
4,13831972,27763544,2131-01-27 04:03:00,2131-01-27 05:39:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Medicaid,ENGLISH,SINGLE,WHITE,2131-01-26 22:19:00,2131-01-27 05:39:00,0
5,18523038,25414328,2142-08-26 17:14:00,2142-08-27 10:00:00,NaN,DIRECT OBSERVATION,PROCEDURE SITE,NaN,Other,ENGLISH,SINGLE,WHITE,NaN,NaN,0
6,16705931,20580522,2174-10-24 11:30:00,2174-10-24 18:45:00,NaN,DIRECT OBSERVATION,PHYSICIAN REFERRAL,NaN,Other,ENGLISH,MARRIED,WHITE,NaN,NaN,0
7,19963742,24951506,2171-07-04 15:58:00,2171-07-05 13:37:00,NaN,AMBULATORY OBSERVATION,PACU,NaN,Other,ENGLISH,SINGLE,UNKNOWN,NaN,NaN,0
8,10903424,22568585,2181-01-31 13:09:00,2181-01-31 13:42:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,NaN,NaN,0
9,15030422,21975876,2149-09-21 23:54:00,2149-09-22 03:59:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,NaN,NaN,0
